In [1]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
import random
import matplotlib.dates as mdates
import math
from plotly import __version__
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode (connected=True)
cf.go_offline()
import datapackage
from Deep_Evolution_Strategy import Deep_Evolution_Strategy
from Model import Model
from Agent import Agent

In [2]:
def HA(df):
    df['HA_Close']=(df['open']+ df['high']+ df['low']+df['close'])/4

    idx = df.index.name
    df.reset_index(inplace=True)

    for i in range(0, len(df)):
        if i == 0:
            df.set_value(i, 'HA_Open', ((df.get_value(i, 'open') + df.get_value(i, 'close')) / 2))
        else:
            df.set_value(i, 'HA_Open', ((df.get_value(i - 1, 'HA_Open') + df.get_value(i - 1, 'HA_Close')) / 2))

    if idx:
        df.set_index(idx, inplace=True)

    df['HA_High']=df[['HA_Open','HA_Close','high']].max(axis=1)
    df['HA_Low']=df[['HA_Open','HA_Close','low']].min(axis=1)
    return df

In [3]:
import time
import datetime
from binance.client import Client
import pandas as pd
import datetime
import dateutil.parser

In [4]:
train_start = datetime.date(2017,12,5)
train_end = datetime.date(2019,12,5)


train_start_unixtime = time.mktime(train_start.timetuple())
train_end_unixtime = time.mktime(train_end.timetuple())


In [5]:
def scrap_historical_binance_price(unit,start,end):

    api_key = 'ddeCyGpnkF2rrO0NyzFjjzFD6fWL4LDurf1e704GqkWck20V777VvxBNUSJkFGmP'
    api_secret = 'oZNSUX3f5Sltzr9KJBkNng8rA5hc14sV45ufNczM5MSRWFMmGcpXcQF6j0dZEkeU'

    client = Client(api_key, api_secret, {"verify": False, "timeout": 20})
    klines = client.get_historical_klines("BTCUSDT", unit, start, end)
    
    data = pd.DataFrame();
    tradingdate = []
    openprice = []
    highprice = []
    lowprice = []
    closeprice = [] 
    for i in range(len(klines)):   
        time = datetime.datetime.utcfromtimestamp(klines[i][0]/1000).strftime('%Y-%m-%d %H:%M:%S')
        format_time = dateutil.parser.parse(time)
        tradingdate.append(format_time)
        openprice.append(float(klines[i][1]))
        highprice.append(float(klines[i][2]))
        lowprice.append(float(klines[i][3]))
        closeprice.append(float(klines[i][4]))
    data['datetime'] = tradingdate
    data['low'] = lowprice
    data['high'] = highprice
    data['open'] = openprice
    data['close'] = closeprice
    data['datetime'] = pd.to_datetime(data['datetime'],unit='s')
    
    '''
    Get VIX index data
    '''
    data_url = 'https://datahub.io/core/finance-vix/datapackage.json'

    # to load Data Package into storage
    package = datapackage.Package(data_url)

    # to load only tabular data
    resources = package.resources
    for resource in resources:
        if resource.tabular:
            vix = pd.read_csv(resource.descriptor['path'])

#     vix = vix[3524:].reset_index(drop = True) # Start from 2018/01/02
    vix['Date'] = pd.to_datetime(vix.Date)
    vix['Date']=vix.Date + datetime.timedelta(days=1)
    vix.rename(columns = {'Date':'datetime','VIX Close': 'VIX_Close'}, inplace = True)
    
    df = pd.merge(data, vix, on='datetime', how='left').fillna(method='ffill')
    return df

In [6]:
# urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
pd.set_option('display.float_format',lambda x : '%.2f' % x)

start = str(int(train_start_unixtime))
end = str(int(train_end_unixtime))

train_df = scrap_historical_binance_price("1d", start, end)
HA(train_df)
train_df.head()

/Users/shinyuwu/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings

/Users/shinyuwu/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings

/Users/shinyuwu/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings

/Users/shinyuwu/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning:

Unverified HTTPS request is being made. A

,index,datetime,low,high,open,close,VIX Open,VIX High,VIX Low,VIX_Close,HA_Close,HA_Open,HA_High,HA_Low
0,0,2018-01-01,12750.00,13818.55,13715.65,13380.00,nan,nan,nan,nan,13416.05,13547.83,13818.55,12750.00
1,1,2018-01-02,12890.02,15473.49,13382.16,14675.11,nan,nan,nan,nan,14105.19,13481.94,15473.49,12890.02
2,2,2018-01-03,14150.00,15307.56,14690.00,14919.51,10.95,11.07,9.52,9.77,14766.77,13793.57,15307.56,13793.57
3,3,2018-01-04,13918.04,15280.00,14919.51,15059.54,9.56,9.65,8.94,9.15,14794.27,14280.17,15280.00,13918.04
4,4,2018-01-05,14600.00,17176.24,15059.56,16960.39,9.01,9.31,8.92,9.22,15949.05,14537.22,17176.24,14537.22


In [7]:
close = train_df.close.values.tolist()
time = train_df.datetime.tolist()
HA_close = train_df.HA_Close.values.tolist()
HA_open = train_df.HA_Open.values.tolist()
HA_high = train_df.HA_High.values.tolist()
HA_low = train_df.HA_Low.values.tolist()
VIX_close = train_df.VIX_Close.values.tolist()
window_size = 51
skip = 1
l = len(close) - 1

In [8]:
model = Model(window_size, 500, 3)
'''
Agent(model, close, money, btc, btcPrice, bnbCoin)

For these parameters, you can choose 
    1) set initial money and let the agent to trade
    2) you buy the bitcoin in advanced, and let the agent to trade. 
       You need to input how many bitcoin you buy, and the price of the bitcoin.
       For example, if you buy 0.05 bitcoin at price $ 8000, you should set btc = 0.05, btcPrice = 400
- model
- close : bitcoin close pricehttp://localhost:8888/notebooks/BitBotBoom/shinyuwu/TradingBot/Real%20Time%20Trade.ipynb#
- money : the initial investment money
- btc: the initial number of bitcoin
- btcPrice: the buying price for the initial number of bitcoin
- bnbCoin : initial bnbcoin

'''
agent = Agent(model,close,time,0, 0.04512284, 500, 0.99932500)
agent.fit(300, 10)

iter 10. reward: -605.695630
iter 20. reward: -402.975239
iter 30. reward: -311.905855


In [9]:
from binance.client import Client
import pandas as pd
import datetime
import dateutil.parser
import time as t
import urllib3
import datapackage
import pandas as pd

api_key = 'ddeCyGpnkF2rrO0NyzFjjzFD6fWL4LDurf1e704GqkWck20V777VvxBNUSJkFGmP'
api_secret = 'oZNSUX3f5Sltzr9KJBkNng8rA5hc14sV45ufNczM5MSRWFMmGcpXcQF6j0dZEkeU'
data_url = 'https://datahub.io/core/finance-vix/datapackage.json'

data = pd.DataFrame();
close = []
time = []
openprice = []
highprice = []
lowprice = []
vix_ = []

last_price = -1

while True:
    
    client = Client(api_key, api_secret, {"verify": False, "timeout": 20})
    klines1 = client.get_historical_klines("BTCUSDT", '1m', "1 minutes ago UTC")
    
    
    # to load Data Package into storage
    package = datapackage.Package(data_url)

    # to load only tabular data
    resources = package.resources
    
    for resource in resources:
        if resource.tabular:
            vix = pd.read_csv(resource.descriptor['path'])
    vix_.append(vix.iloc[-1]['VIX Close'])

    for i in range(len(klines1)): 
        ha_high = []
        ha_low = []
        
        urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
    
        close.append(float(klines1[i][4]))
        openprice.append(float(klines1[i][1]))
        highprice.append(float(klines1[i][2]))
        lowprice.append(float(klines1[i][3]))
        
        ha_high.extend([float(klines1[i][4]), float(klines1[i][1]), float(klines1[i][2])])
        ha_low.extend([float(klines1[i][4]), float(klines1[i][1]), float(klines1[i][3])])
        
        HA_Close = (close[i]+openprice[i]+highprice[i]+close[i])/4
        if i == 0:
            HA_Open=(openprice[i] + close[i]) / 2
        else:
            HA_Open=(openprice[i-1] + close[i-1]) / 2
            
        HA_High = max(ha_high)
        HA_Low =  min(ha_low)
        now = datetime.datetime.now()
        
    now.strftime("%Y-%m-%d %H:%M")
    time.append(now.isoformat())
    print(now.isoformat(), 'BTC price = ',float(klines1[i][4]))
    print(close)
    window_size = 51
    skip = 1
    l = len(close)-1
    realtimeAgent = Agent(model,close,time,0, 0.04512284, 500, 0.99932500)
    realtimeAgent.buy(HA_close,HA_open,HA_high,HA_low, vix_)
#     print(now.isoformat(), 'BTC price = ',close[-1])
#     print(close)
    print('#######################################')
    t.sleep(86400)

/Users/shinyuwu/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings

/Users/shinyuwu/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings

/Users/shinyuwu/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings



2019-12-06T14:08:18.139930 BTC price =  7407.98
[7407.98]

total gained -500.000000, total investment -100.000000 %


#######################################


KeyboardInterrupt: 